# OD Analysis and Interactive Map Visualizations

This section provides interactive map visualizations for:

1. **OD Analysis within Bangkok (BKK ↔ BKK)**
2. **OD Analysis between Bangkok and Metropolitan Area (BKK ↔ Metro)**
3. **OD Analysis within Metropolitan Area (Metro ↔ Metro, excluding BKK)**

Each block below will show the top OD pairs on an interactive map using Plotly.

In [4]:
!pip install pandas numpy matplotlib seaborn scikit-learn jupyterlab folium plotly pyspark 

In [5]:
import pyspark.sql.functions as F
from pyspark.sql import Window
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
from pyspark.sql.functions import month
import os
warnings.filterwarnings('ignore')

os.environ["PYSPARK_SUBMIT_ARGS"] = "--driver-memory 8g --executor-memory 8g pyspark-shell"

In [6]:
from pyspark.sql import SparkSession


# Initialize Spark session
spark = SparkSession.builder \
    .appName("TaxiDataODAnalysis") \
    .getOrCreate()

# Path to processed parquet output
parquet_path = "taxi_processed_tourist.parquet"

# Read the processed parquet data
df = spark.read.parquet(parquet_path)

# Show schema and a sample of the data
print("Schema:")
df.printSchema()

print("\nSample data:")
df.show(5, truncate=False)

# Optionally, count records
print(f"\nTotal records: {df.count():,}")

Schema:
root
 |-- vehicle_id: string (nullable = true)
 |-- date_only: date (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- heading: integer (nullable = true)
 |-- for_hire_light: integer (nullable = true)
 |-- engine_acc: integer (nullable = true)
 |-- gpsvalid: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- day_name: string (nullable = true)
 |-- is_weekend: integer (nullable = true)
 |-- time_of_day: string (nullable = true)
 |-- day_night_shift: string (nullable = true)
 |-- is_rush_hour: integer (nullable = true)
 |-- is_hired: integer (nullable = true)
 |-- is_searching_fare: integer (nullable = true)
 |-- is_idle: integer (nullable = true)
 |-- is_moving: integer (nullable = true)
 |-- is_stationary: integer (nullable = true)
 |-- speed_category: string (nullable = true)
 |-- distance_from_b

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, when, desc
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel

# Initialize Spark Session
# NOTE: This assumes the SparkSession is already active or initialized in your environment.

# --- PERFORMANCE OPTIMIZATION SETTINGS ---
# These settings can help reduce shuffle spill and improve parallelism.
# Only apply if you are initializing the Spark Session here.
try:
    spark = SparkSession.builder.appName("OptimizedTaxiODAnalysis").config("spark.sql.shuffle.partitions", "200").config("spark.driver.maxResultSize", "4g").getOrCreate()
except NameError:
    # If spark session is not defined, handle the error as before
    print("WARNING: Spark session or DataFrame 'df' is not defined. Skipping analysis.")
    exit()

# Define the provinces for the analysis
PROVINCE_BKK = "Bangkok"
PROVINCES_METRO_AREA = ["Nonthaburi", "Pathum Thani", "Samut Prakan", "Samut Sakhon", "Nakhon Pathom"]
PROVINCES_BMR = [PROVINCE_BKK] + PROVINCES_METRO_AREA

# --- DATA ASSUMPTION ---
# The DataFrame 'df' with the provided schema is assumed to be available here.
print(f"Starting OD Analysis based on the provided schema.")
print("-" * 50)

# --- 1. TRIP RECONSTRUCTION AND OD PAIRING (Optimized) ---

# 1.1. Define Window for trip grouping
# Partition by vehicle_id and order by timestamp to correctly sequence the pings.
window_spec = Window.partitionBy("vehicle_id").orderBy("timestamp")

# 1.2. Create a unique Trip ID: A trip starts when trip_start=1.
# This cumulative sum is the most expensive operation.
df_with_trip_id = df.withColumn(
    "trip_id",
    sum(col("trip_start")).over(window_spec).cast("long")
).filter(col("trip_id") > 0)

# 1.3. CACHING: Persist the result of the expensive window operation in memory (if space allows).
# This avoids recalculating the trip_id for the next three steps.
df_with_trip_id.persist(StorageLevel.MEMORY_AND_DISK)
# Force materialization/computation now
df_with_trip_id.count()


# 1.4. Extract Origin and Destination details and Aggregate metrics from the cached DF

# Origin (O) - Filter for start records
df_origin = df_with_trip_id.filter(col("trip_start") == 1) \
    .select(
        col("trip_id"),
        col("vehicle_id"),
        col("nearest_province").alias("O_province"),
        col("grid_cell").alias("O_grid_cell"),
        col("timestamp").alias("O_timestamp")
    )

# Destination (D) - Filter for end records
df_destination = df_with_trip_id.filter(col("trip_end") == 1) \
    .select(
        col("trip_id"),
        col("nearest_province").alias("D_province"),
        col("grid_cell").alias("D_grid_cell"),
        col("timestamp").alias("D_timestamp")
    )

# Aggregate total distance and number of pings per trip
# This is a large aggregation but benefits from the cached df_with_trip_id.
trip_metrics = df_with_trip_id.groupBy("trip_id").agg(
    sum("distance_traveled_km").alias("trip_distance_km"),
    count("*").alias("ping_count")
)

# Join O, D, and Metrics to form the final OD Trips DataFrame
# The join key "trip_id" is unique for O, D, and Metrics, making the final distinct() unnecessary.
df_od_trips = df_origin.join(df_destination, "trip_id", "inner") \
    .join(trip_metrics, "trip_id", "inner")

print(f"Total reconstructed OD Trips (unique pairs): {df_od_trips.count()}")
df_od_trips.select("trip_id", "O_province", "D_province", "trip_distance_km").limit(5).show(truncate=False)
print("-" * 50)


Starting OD Analysis based on the provided schema.
--------------------------------------------------


25/11/30 05:25:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Total reconstructed OD Trips (unique pairs): 297369329


+-------+------------+------------+-----------------+
|trip_id|O_province  |D_province  |trip_distance_km |
+-------+------------+------------+-----------------+
|26     |Nonthaburi  |Samut Sakhon|49654.01144188893|
|26     |Samut Prakan|Samut Sakhon|49654.01144188893|
|26     |Samut Prakan|Samut Sakhon|49654.01144188893|
|26     |Bangkok     |Samut Sakhon|49654.01144188893|
|26     |Bangkok     |Samut Sakhon|49654.01144188893|
+-------+------------+------------+-----------------+

--------------------------------------------------


In [8]:
# --- 2. OD ANALYSIS BLOCKS ---

# 2.1. OD Analysis within Bangkok (BKK <-> BKK)
df_bkk_bkk = df_od_trips.filter(
    (col("O_province") == PROVINCE_BKK) & (col("D_province") == PROVINCE_BKK)
)

print("--- BLOCK 1: OD ANALYSIS WITHIN BANGKOK (BKK <-> BKK) ---")

bkk_bkk_summary = df_bkk_bkk.agg(
    count("*").alias("Total_Trips"),
    sum("trip_distance_km").alias("Total_Distance_km"),
    (sum("trip_distance_km") / count("*")).alias("Avg_Trip_Distance_km")
)
bkk_bkk_summary.show()

# Top 5 Intra-BKK Grid-to-Grid Routes (finer analysis)
bkk_bkk_detail = df_bkk_bkk.groupBy("O_grid_cell", "D_grid_cell").agg(
    count("*").alias("Trip_Count"),
    (sum("trip_distance_km") / count("*")).alias("Avg_Distance_km")
).orderBy(desc("Trip_Count"))

print("Top 5 Intra-Bangkok Grid-to-Grid OD Pairs:")
bkk_bkk_detail.limit(5).show(truncate=False)
print("-" * 50)


# 2.2. OD Analysis between Bangkok and Metropolitan Area (BKK <-> BMR)
df_bkk_bmr = df_od_trips.filter(
    (
        (col("O_province") == PROVINCE_BKK) & (col("D_province").isin(PROVINCES_METRO_AREA))
    ) | (
        (col("D_province") == PROVINCE_BKK) & (col("O_province").isin(PROVINCES_METRO_AREA))
    )
).withColumn(
    "Flow_Direction",
    when(col("O_province") == PROVINCE_BKK, "BKK -> Metro")
    .otherwise("Metro -> BKK")
)

print("--- BLOCK 2: OD ANALYSIS BETWEEN BANGKOK AND METROPOLITAN AREA ---")

# Summary by Flow Direction (Inbound vs Outbound)
bkk_bmr_summary = df_bkk_bmr.groupBy("Flow_Direction").agg(
    count("*").alias("Total_Trips"),
    (sum("trip_distance_km") / count("*")).alias("Avg_Trip_Distance_km")
)
bkk_bmr_summary.show()

# Detailed analysis by specific Metropolitan province (Top 5 inter-provincial pairs)
bkk_bmr_detail = df_bkk_bmr.groupBy("O_province", "D_province").agg(
    count("*").alias("Total_Trips")
).orderBy(desc("Total_Trips"))

print("Top 5 BKK <-> Metropolitan OD Pairs:")
bkk_bmr_detail.limit(5).show(truncate=False)
print("-" * 50)


# 2.3. OD Analysis within the Metropolitan Area (BMR <-> BMR excluding BKK)
df_bmr_bmr = df_od_trips.filter(
    (col("O_province").isin(PROVINCES_METRO_AREA)) & (col("D_province").isin(PROVINCES_METRO_AREA))
)

print("--- BLOCK 3: OD ANALYSIS WITHIN METROPOLITAN AREA (Excluding BKK) ---")

bmr_bmr_summary = df_bmr_bmr.agg(
    count("*").alias("Total_Trips"),
    (sum("trip_distance_km") / count("*")).alias("Avg_Trip_Distance_km")
)
bmr_bmr_summary.show()

# Detailed BMR <-> BMR OD pairs (e.g., Nonthaburi -> Pathum Thani)
bmr_bmr_detail = df_bmr_bmr.groupBy("O_province", "D_province").agg(
    count("*").alias("Total_Trips")
).orderBy(desc("Total_Trips"))

print("Top 5 BMR <-> BMR OD Pairs:")
bmr_bmr_detail.limit(5).show(truncate=False)

# Don't forget to unpersist the cached DataFrame when you are done with all analysis.
# df_with_trip_id.unpersist()

--- BLOCK 1: OD ANALYSIS WITHIN BANGKOK (BKK <-> BKK) ---


+-----------+--------------------+--------------------+
|Total_Trips|   Total_Distance_km|Avg_Trip_Distance_km|
+-----------+--------------------+--------------------+
|   61177711|2.649146405004917E12|   43302.47669784371|
+-----------+--------------------+--------------------+

Top 5 Intra-Bangkok Grid-to-Grid OD Pairs:


+-----------+------------+----------+------------------+
|O_grid_cell|D_grid_cell |Trip_Count|Avg_Distance_km   |
+-----------+------------+----------+------------------+
|13.76_100.5|13.76_100.5 |37084     |42890.495882219366|
|13.76_100.5|13.76_100.48|30479     |41788.21147669597 |
|13.8_100.55|13.76_100.5 |27596     |41986.377986082225|
|13.76_100.5|13.75_100.5 |25189     |43376.52461113319 |
|13.76_100.5|13.8_100.55 |24934     |42987.9282925005  |
+-----------+------------+----------+------------------+

--------------------------------------------------
--- BLOCK 2: OD ANALYSIS BETWEEN BANGKOK AND METROPOLITAN AREA ---


+--------------+-----------+--------------------+
|Flow_Direction|Total_Trips|Avg_Trip_Distance_km|
+--------------+-----------+--------------------+
|  Metro -> BKK|   77267005|   43602.38509742801|
|  BKK -> Metro|   70156900|   43827.77130248205|
+--------------+-----------+--------------------+

Top 5 BKK <-> Metropolitan OD Pairs:


+------------+------------+-----------+
|O_province  |D_province  |Total_Trips|
+------------+------------+-----------+
|Nonthaburi  |Bangkok     |40022443   |
|Bangkok     |Nonthaburi  |37594545   |
|Samut Prakan|Bangkok     |24755077   |
|Bangkok     |Samut Prakan|22139293   |
|Pathum Thani|Bangkok     |8358697    |
+------------+------------+-----------+

--------------------------------------------------
--- BLOCK 3: OD ANALYSIS WITHIN METROPOLITAN AREA (Excluding BKK) ---


+-----------+--------------------+
|Total_Trips|Avg_Trip_Distance_km|
+-----------+--------------------+
|   88767713|   44099.16016848852|
+-----------+--------------------+

Top 5 BMR <-> BMR OD Pairs:


+------------+------------+-----------+
|O_province  |D_province  |Total_Trips|
+------------+------------+-----------+
|Nonthaburi  |Nonthaburi  |24633143   |
|Samut Prakan|Nonthaburi  |15210083   |
|Nonthaburi  |Samut Prakan|14450356   |
|Samut Prakan|Samut Prakan|9035194    |
|Pathum Thani|Nonthaburi  |5136124    |
+------------+------------+-----------+



In [9]:
# --- BKK <-> BMR Grid-to-Grid OD Pairs ---
# Filter for BKK <-> BMR trips (Bangkok to Metro Area and vice versa)
df_bkk_bmr_grid = df_od_trips.filter(
    ((col("O_province") == PROVINCE_BKK) & (col("D_province").isin(PROVINCES_METRO_AREA))) |
    ((col("D_province") == PROVINCE_BKK) & (col("O_province").isin(PROVINCES_METRO_AREA)))
)

bkk_bmr_grid_detail = df_bkk_bmr_grid.groupBy("O_grid_cell", "D_grid_cell").agg(
    count("*").alias("Trip_Count"),
    (sum("trip_distance_km") / count("*")).alias("Avg_Distance_km")
).orderBy(desc("Trip_Count"))

print("Top 5 BKK <-> BMR Grid-to-Grid OD Pairs:")
bkk_bmr_grid_detail.limit(5).show(truncate=False)

# --- BMR <-> BMR Grid-to-Grid OD Pairs (excluding BKK) ---
df_bmr_bmr_grid = df_od_trips.filter(
    (col("O_province").isin(PROVINCES_METRO_AREA)) & (col("D_province").isin(PROVINCES_METRO_AREA))
)

bmr_bmr_grid_detail = df_bmr_bmr_grid.groupBy("O_grid_cell", "D_grid_cell").agg(
    count("*").alias("Trip_Count"),
    (sum("trip_distance_km") / count("*")).alias("Avg_Distance_km")
).orderBy(desc("Trip_Count"))

print("Top 5 BMR <-> BMR Grid-to-Grid OD Pairs:")
bmr_bmr_grid_detail.limit(5).show(truncate=False)


Top 5 BKK <-> BMR Grid-to-Grid OD Pairs:


+------------+------------+----------+------------------+
|O_grid_cell |D_grid_cell |Trip_Count|Avg_Distance_km   |
+------------+------------+----------+------------------+
|13.76_100.5 |13.92_100.6 |32598     |45086.43175116635 |
|13.92_100.6 |13.76_100.5 |30157     |42635.972616090265|
|13.76_100.5 |13.81_100.56|28015     |43893.80667585524 |
|13.92_100.6 |13.76_100.48|25480     |41494.60809642727 |
|13.81_100.55|13.76_100.5 |25287     |41895.115211001794|
+------------+------------+----------+------------------+

Top 5 BMR <-> BMR Grid-to-Grid OD Pairs:


+------------+------------+----------+------------------+
|O_grid_cell |D_grid_cell |Trip_Count|Avg_Distance_km   |
+------------+------------+----------+------------------+
|13.92_100.6 |13.92_100.6 |27194     |44727.02019729382 |
|13.92_100.6 |13.81_100.56|23501     |43399.820582943605|
|13.81_100.55|13.92_100.6 |22154     |44192.55735217874 |
|13.69_100.75|13.92_100.6 |21789     |47123.2261534702  |
|13.92_100.6 |13.69_100.75|20145     |49428.16827733199 |
+------------+------------+----------+------------------+



## Interactive OD Map Visualizations with Plotly

In [16]:
import folium
from folium.plugins import MarkerCluster, MiniMap
import pandas as pd
import numpy as np

# --- 1. Data Mockup (REPLACE with your actual Spark code) ---
# NOTE: This section is for demonstration. Keep your original Spark data loading:
top_od_pairs = bkk_bkk_detail.limit(20).toPandas()

# --- 2. Coordinate Parsing Function (CRITICAL) ---

def get_coords_from_grid_cell(grid_cell_id):
    """
    Parses the 'lat_lon' string format (e.g., '13.75_100.54') 
    to return (lat, lon) tuple.
    """
    if isinstance(grid_cell_id, str) and '_' in grid_cell_id:
        try:
            lat_str, lon_str = grid_cell_id.split('_')
            # Folium expects (lat, lon) for positioning
            return float(lat_str), float(lon_str)
        except ValueError:
            return np.nan, np.nan
    return np.nan, np.nan

# --- 3. Apply Coordinates to DataFrame ---

# Apply the function to create the new coordinate columns based on the grid names
top_od_pairs['O_lat'], top_od_pairs['O_lon'] = zip(*top_od_pairs['O_grid_cell'].apply(get_coords_from_grid_cell))
top_od_pairs['D_lat'], top_od_pairs['D_lon'] = zip(*top_od_pairs['D_grid_cell'].apply(get_coords_from_grid_cell))

# Filter out any rows where coordinates could not be parsed
top_od_pairs.dropna(subset=['O_lat', 'D_lat'], inplace=True)

# --- 3.5. Sort Data by Trip_Count (NEW/MODIFIED) ---
# Sort the DataFrame in descending order based on 'Trip_Count'. 
# This ensures that Route 1 is the route with the highest traffic.
top_od_pairs.sort_values(by='Trip_Count', ascending=False, inplace=True)

# --- 4. Folium Visualization Setup ---

# Calculate the center of the data for map initialization
if not top_od_pairs.empty:
    center_lat = top_od_pairs[['O_lat', 'D_lat']].mean().mean()
    center_lon = top_od_pairs[['O_lon', 'D_lon']].mean().mean()
else:
    # Default center if data is empty (e.g., Bangkok coordinates)
    center_lat, center_lon = 13.737, 100.535

# Initialize the Folium Map with a professional tile set
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=12,
    tiles='CartoDB Positron' # A clean, professional-looking tile set
)

# Add a MiniMap for better navigation on a large map
MiniMap().add_to(m)

# Create feature groups
destination_clusters = MarkerCluster(name="Destination Hotspots", icon_create_function=None).add_to(m)
flow_layer = folium.FeatureGroup(name="OD Flows and Origins").add_to(m)
poi_layer = folium.FeatureGroup(name="Points of Interest (POI)").add_to(m)

# Define the scaling factor for line weight
max_trip_count = top_od_pairs['Trip_Count'].max()
MIN_LINE_WEIGHT = 2
MAX_LINE_WEIGHT = 10

# Color palette for unique route distinction (up to 10 distinct colors)
COLOR_PALETTE = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'
 ]

# Assign a unique color and a descriptive label to each row
# Since the data is sorted, Route 1 will be the top trip count
top_od_pairs['Color'] = [COLOR_PALETTE[i % len(COLOR_PALETTE)] for i in range(len(top_od_pairs))]
top_od_pairs['Route_ID'] = [f"Route {i+1}" for i in range(len(top_od_pairs))]


# --- POI Data Mockup ---
# Assuming the data is in Bangkok area
pois = [
    {'name': 'Grand Palace', 'lat': 13.7494, 'lon': 100.4938, 'icon': 'fa-building'},
    {'name': 'Siam Paragon', 'lat': 13.7460, 'lon': 100.5348, 'icon': 'fa-shopping-bag'},
    {'name': 'Chatuchak Market', 'lat': 13.8038, 'lon': 100.5512, 'icon': 'fa-store'},
    {'name': 'Iconsiam', 'lat': 13.7260, 'lon': 100.5108, 'icon': 'fa-shopping-bag'},
    {'name': 'Wat Arun', 'lat': 13.7437, 'lon': 100.4880, 'icon': 'fa-university'},
    {'name': 'Khao San Road', 'lat': 13.7597, 'lon': 100.4977, 'icon': 'fa-road'},
    {'name': 'Suvarnabhumi Airport', 'lat': 13.6900, 'lon': 100.7501, 'icon': 'fa-plane'},
    {'name': 'Don Mueang Airport', 'lat': 13.9138, 'lon': 100.6015, 'icon': 'fa-plane'}
]

# Add POIs to the map
for poi in pois:
    folium.Marker(
        location=[poi['lat'], poi['lon']],
        tooltip=poi['name'],
        icon=folium.Icon(color='blue', icon=poi['icon'], prefix='fa')
    ).add_to(poi_layer)


# --- 5. Add Flows and Markers to the Map ---

for index, row in top_od_pairs.iterrows():
    O_coord = (row['O_lat'], row['O_lon'])
    D_coord = (row['D_lat'], row['D_lon'])
    trip_count = row['Trip_Count']
    route_color = row['Color']
    route_id = row['Route_ID']

    # Scale line weight by flow intensity
    flow_intensity = trip_count / max_trip_count
    line_weight = max(MIN_LINE_WEIGHT, flow_intensity * MAX_LINE_WEIGHT)

    popup_text = f"""
    <b>{route_id}</b><br>
    <b>Trip Count: {trip_count:,}</b><br>
    Origin: {row['O_grid_cell']}<br>
    Destination: {row['D_grid_cell']}
    """
    
    # 1. Draw the OD Flow Line
    folium.PolyLine(
        locations=[O_coord, D_coord],
        weight=line_weight,
        color=route_color, 
        opacity=0.8,
        tooltip=f"{route_id} ({trip_count} Trips)"
    ).add_to(flow_layer)

    # 2. Add Origin Marker (O) 
    folium.CircleMarker(
        location=O_coord,
        radius=5,
        color=route_color,
        fill=True,
        fill_color=route_color,
        fill_opacity=1,
        tooltip=f"{route_id} Origin: {row['O_grid_cell']}"
    ).add_to(flow_layer)

    # 3. Add Destination Marker (D) to the MarkerCluster 
    folium.Marker(
        location=D_coord,
        icon=folium.Icon(color='red', icon='fa-bullseye', prefix='fa'), 
        popup=popup_text
    ).add_to(destination_clusters)


# --- 5.5 Add Custom HTML Legend ---

# The legend will use the route IDs and colors, ordered by most frequent route
legend_html = """
     <div style=\"position: fixed; 
                 bottom: 50px; left: 50px; width: 180px; height: auto; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white; opacity:0.9;\">
       &nbsp; <b>Top Routes (Most to Least)</b> <br>
       """
# Add a line for each unique route color and ID, ordered by Trip_Count
for i, row in top_od_pairs.iterrows():
    legend_html += f"""
       &nbsp; <i style='background:{row['Color']}; border-radius:50%; width:10px; height:10px; display:inline-block;'></i> &nbsp; {row['Route_ID']} ({row['Trip_Count']:,} trips)<br>
       """

# Add a section for Line Weight legend
legend_html += f"""
       <hr style=\"margin:5px 0;\">
       &nbsp; <b>Flow Intensity (Weight)</b> <br>
       &nbsp; <i style=\"background:grey; width:10px; height:1px; display:inline-block;\"></i> &nbsp; Low Trips <br>
       &nbsp; <i style=\"background:grey; width:10px; height:5px; display:inline-block;\"></i> &nbsp; Medium Trips <br>
       &nbsp; <i style=\"background:grey; width:10px; height:{MAX_LINE_WEIGHT}px; display:inline-block;\"></i> &nbsp; High Trips <br>
       </div>
       """

m.get_root().html.add_child(folium.Element(legend_html))


# Add a custom HTML title to the map
title_html = f'''
             <h3 align=\"center\" style=\"font-size:18px; font-weight:bold; margin-top:5px; margin-bottom:5px;\">
                Top 20 Most Frequent OD Pairs within Bangkok (BKK &rarr; BKK)
             </h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# Add a LayerControl to toggle flow, cluster, and POI visibility
folium.LayerControl().add_to(m)

# --- 6. Save to HTML ---
output_filename = "OD_Most_20_BKK_BKK.html"
m.save(output_filename)
print(f"Interactive Folium map saved as {output_filename}")

Interactive Folium map saved as OD_Most_20_BKK_BKK.html


In [14]:
import folium
from folium.plugins import MarkerCluster, MiniMap
import pandas as pd
import numpy as np

# --- 1. Data Mockup (REPLACE with your actual Spark code) ---
# NOTE: This section is for demonstration. Keep your original Spark data loading:
top_od_pairs = bkk_bmr_grid_detail.limit(20).toPandas()  # Use grid-to-grid detail for most route analysis

# --- 2. Coordinate Parsing Function (CRITICAL) ---

def get_coords_from_grid_cell(grid_cell_id):
    """
    Parses the 'lat_lon' string format (e.g., '13.75_100.54') 
    to return (lat, lon) tuple.
    """
    if isinstance(grid_cell_id, str) and '_' in grid_cell_id:
        try:
            lat_str, lon_str = grid_cell_id.split('_')
            # Folium expects (lat, lon) for positioning
            return float(lat_str), float(lon_str)
        except ValueError:
            return np.nan, np.nan
    return np.nan, np.nan

# --- 3. Apply Coordinates to DataFrame ---

# Apply the function to create the new coordinate columns based on the grid names
top_od_pairs['O_lat'], top_od_pairs['O_lon'] = zip(*top_od_pairs['O_grid_cell'].apply(get_coords_from_grid_cell))
top_od_pairs['D_lat'], top_od_pairs['D_lon'] = zip(*top_od_pairs['D_grid_cell'].apply(get_coords_from_grid_cell))

# Filter out any rows where coordinates could not be parsed
top_od_pairs.dropna(subset=['O_lat', 'D_lat'], inplace=True)

# --- 3.5. Sort Data by Trip_Count (NEW/MODIFIED) ---
# Sort the DataFrame in descending order based on 'Trip_Count'. 
# This ensures that Route 1 is the route with the highest traffic.
top_od_pairs.sort_values(by='Trip_Count', ascending=False, inplace=True)

# --- 4. Folium Visualization Setup ---

# Calculate the center of the data for map initialization
if not top_od_pairs.empty:
    center_lat = top_od_pairs[['O_lat', 'D_lat']].mean().mean()
    center_lon = top_od_pairs[['O_lon', 'D_lon']].mean().mean()
else:
    # Default center if data is empty (e.g., Bangkok coordinates)
    center_lat, center_lon = 13.737, 100.535

# Initialize the Folium Map with a professional tile set
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=12,
    tiles='CartoDB Positron' # A clean, professional-looking tile set
)

# Add a MiniMap for better navigation on a large map
MiniMap().add_to(m)

# Create feature groups
destination_clusters = MarkerCluster(name="Destination Hotspots", icon_create_function=None).add_to(m)
flow_layer = folium.FeatureGroup(name="OD Flows and Origins").add_to(m)
poi_layer = folium.FeatureGroup(name="Points of Interest (POI)").add_to(m)

# Define the scaling factor for line weight
max_trip_count = top_od_pairs['Trip_Count'].max()
MIN_LINE_WEIGHT = 2
MAX_LINE_WEIGHT = 10

# Color palette for unique route distinction (up to 10 distinct colors)
COLOR_PALETTE = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'
 ]

# Assign a unique color and a descriptive label to each row
# Since the data is sorted, Route 1 will be the top trip count
top_od_pairs['Color'] = [COLOR_PALETTE[i % len(COLOR_PALETTE)] for i in range(len(top_od_pairs))]
top_od_pairs['Route_ID'] = [f"Route {i+1}" for i in range(len(top_od_pairs))]


# --- POI Data Mockup ---
# Assuming the data is in Bangkok area
pois = [
    {'name': 'Grand Palace', 'lat': 13.7494, 'lon': 100.4938, 'icon': 'fa-building'},
    {'name': 'Siam Paragon', 'lat': 13.7460, 'lon': 100.5348, 'icon': 'fa-shopping-bag'},
    {'name': 'Chatuchak Market', 'lat': 13.8038, 'lon': 100.5512, 'icon': 'fa-store'},
    {'name': 'Iconsiam', 'lat': 13.7260, 'lon': 100.5108, 'icon': 'fa-shopping-bag'},
    {'name': 'Wat Arun', 'lat': 13.7437, 'lon': 100.4880, 'icon': 'fa-university'},
    {'name': 'Khao San Road', 'lat': 13.7597, 'lon': 100.4977, 'icon': 'fa-road'},
    {'name': 'Suvarnabhumi Airport', 'lat': 13.6900, 'lon': 100.7501, 'icon': 'fa-plane'},
    {'name': 'Don Mueang Airport', 'lat': 13.9138, 'lon': 100.6015, 'icon': 'fa-plane'}
]

# Add POIs to the map
for poi in pois:
    folium.Marker(
        location=[poi['lat'], poi['lon']],
        tooltip=poi['name'],
        icon=folium.Icon(color='blue', icon=poi['icon'], prefix='fa')
    ).add_to(poi_layer)


# --- 5. Add Flows and Markers to the Map ---

for index, row in top_od_pairs.iterrows():
    O_coord = (row['O_lat'], row['O_lon'])
    D_coord = (row['D_lat'], row['D_lon'])
    trip_count = row['Trip_Count']
    route_color = row['Color']
    route_id = row['Route_ID']

    # Scale line weight by flow intensity
    flow_intensity = trip_count / max_trip_count
    line_weight = max(MIN_LINE_WEIGHT, flow_intensity * MAX_LINE_WEIGHT)

    popup_text = f"""
    <b>{route_id}</b><br>
    <b>Trip Count: {trip_count:,}</b><br>
    Origin: {row['O_grid_cell']}<br>
    Destination: {row['D_grid_cell']}
    """
    
    # 1. Draw the OD Flow Line
    folium.PolyLine(
        locations=[O_coord, D_coord],
        weight=line_weight,
        color=route_color, 
        opacity=0.8,
        tooltip=f"{route_id} ({trip_count} Trips)"
    ).add_to(flow_layer)

    # 2. Add Origin Marker (O) 
    folium.CircleMarker(
        location=O_coord,
        radius=5,
        color=route_color,
        fill=True,
        fill_color=route_color,
        fill_opacity=1,
        tooltip=f"{route_id} Origin: {row['O_grid_cell']}"
    ).add_to(flow_layer)

    # 3. Add Destination Marker (D) to the MarkerCluster 
    folium.Marker(
        location=D_coord,
        icon=folium.Icon(color='red', icon='fa-bullseye', prefix='fa'), 
        popup=popup_text
    ).add_to(destination_clusters)


# --- 5.5 Add Custom HTML Legend ---

# The legend will use the route IDs and colors, ordered by most frequent route
legend_html = """
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 180px; height: auto; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white; opacity:0.9;">
       &nbsp; <b>Top Routes (Most to Least)</b> <br>
       """
# Add a line for each unique route color and ID, ordered by Trip_Count
for i, row in top_od_pairs.iterrows():
    legend_html += f"""
       &nbsp; <i style='background:{row['Color']}; border-radius:50%; width:10px; height:10px; display:inline-block;'></i> &nbsp; {row['Route_ID']} ({row['Trip_Count']:,} trips)<br>
       """

# Add a section for Line Weight legend
legend_html += f"""
       <hr style="margin:5px 0;">
       &nbsp; <b>Flow Intensity (Weight)</b> <br>
       &nbsp; <i style="background:grey; width:10px; height:1px; display:inline-block;"></i> &nbsp; Low Trips <br>
       &nbsp; <i style="background:grey; width:10px; height:5px; display:inline-block;"></i> &nbsp; Medium Trips <br>
       &nbsp; <i style="background:grey; width:10px; height:{MAX_LINE_WEIGHT}px; display:inline-block;"></i> &nbsp; High Trips <br>
       </div>
       """

m.get_root().html.add_child(folium.Element(legend_html))


# Add a custom HTML title to the map
title_html = f'''
             <h3 align="center" style="font-size:18px; font-weight:bold; margin-top:5px; margin-bottom:5px;">
             Most Frequent BKK <-> BMR Grid-to-Grid Taxi Routes
             </h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# Add a LayerControl to toggle flow, cluster, and POI visibility
folium.LayerControl().add_to(m)

# --- 6. Save to HTML ---
output_filename = "OD_Most_20_BKK_BMR.html"
m.save(output_filename)
print(f"Interactive Folium map saved as {output_filename}")

Interactive Folium map saved as OD_Most_20_BKK_BMR.html


In [ ]:
import folium
from folium.plugins import MarkerCluster, MiniMap
import pandas as pd
import numpy as np

# --- 1. Data Mockup (REPLACE with your actual Spark code) ---
# NOTE: This section is for demonstration. Keep your original Spark data loading:
top_od_pairs = bmr_bmr_grid_detail.limit(20).toPandas()  # Use grid-to-grid detail for most route analysis

# --- 2. Coordinate Parsing Function (CRITICAL) ---

def get_coords_from_grid_cell(grid_cell_id):
    """
    Parses the 'lat_lon' string format (e.g., '13.75_100.54') 
    to return (lat, lon) tuple.
    """
    if isinstance(grid_cell_id, str) and '_' in grid_cell_id:
        try:
            lat_str, lon_str = grid_cell_id.split('_')
            # Folium expects (lat, lon) for positioning
            return float(lat_str), float(lon_str)
        except ValueError:
            return np.nan, np.nan
    return np.nan, np.nan

# --- 3. Apply Coordinates to DataFrame ---

# Apply the function to create the new coordinate columns based on the grid names
top_od_pairs['O_lat'], top_od_pairs['O_lon'] = zip(*top_od_pairs['O_grid_cell'].apply(get_coords_from_grid_cell))
top_od_pairs['D_lat'], top_od_pairs['D_lon'] = zip(*top_od_pairs['D_grid_cell'].apply(get_coords_from_grid_cell))

# Filter out any rows where coordinates could not be parsed
top_od_pairs.dropna(subset=['O_lat', 'D_lat'], inplace=True)

# --- 3.5. Sort Data by Trip_Count (NEW/MODIFIED) ---
# Sort the DataFrame in descending order based on 'Trip_Count'. 
# This ensures that Route 1 is the route with the highest traffic.
top_od_pairs.sort_values(by='Trip_Count', ascending=False, inplace=True)

# --- 4. Folium Visualization Setup ---

# Calculate the center of the data for map initialization
if not top_od_pairs.empty:
    center_lat = top_od_pairs[['O_lat', 'D_lat']].mean().mean()
    center_lon = top_od_pairs[['O_lon', 'D_lon']].mean().mean()
else:
    # Default center if data is empty (e.g., Bangkok coordinates)
    center_lat, center_lon = 13.737, 100.535

# Initialize the Folium Map with a professional tile set
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=12,
    tiles='CartoDB Positron' # A clean, professional-looking tile set
)

# Add a MiniMap for better navigation on a large map
MiniMap().add_to(m)

# Create feature groups
destination_clusters = MarkerCluster(name="Destination Hotspots", icon_create_function=None).add_to(m)
flow_layer = folium.FeatureGroup(name="OD Flows and Origins").add_to(m)
poi_layer = folium.FeatureGroup(name="Points of Interest (POI)").add_to(m)

# Define the scaling factor for line weight
max_trip_count = top_od_pairs['Trip_Count'].max()
MIN_LINE_WEIGHT = 2
MAX_LINE_WEIGHT = 10

# Color palette for unique route distinction (up to 10 distinct colors)
COLOR_PALETTE = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'
 ]

# Assign a unique color and a descriptive label to each row
# Since the data is sorted, Route 1 will be the top trip count
top_od_pairs['Color'] = [COLOR_PALETTE[i % len(COLOR_PALETTE)] for i in range(len(top_od_pairs))]
top_od_pairs['Route_ID'] = [f"Route {i+1}" for i in range(len(top_od_pairs))]


# --- POI Data Mockup ---
# Assuming the data is in Bangkok area
pois = [
    {'name': 'Grand Palace', 'lat': 13.7494, 'lon': 100.4938, 'icon': 'fa-building'},
    {'name': 'Siam Paragon', 'lat': 13.7460, 'lon': 100.5348, 'icon': 'fa-shopping-bag'},
    {'name': 'Chatuchak Market', 'lat': 13.8038, 'lon': 100.5512, 'icon': 'fa-store'},
    {'name': 'Iconsiam', 'lat': 13.7260, 'lon': 100.5108, 'icon': 'fa-shopping-bag'},
    {'name': 'Wat Arun', 'lat': 13.7437, 'lon': 100.4880, 'icon': 'fa-university'},
    {'name': 'Khao San Road', 'lat': 13.7597, 'lon': 100.4977, 'icon': 'fa-road'},
    {'name': 'Suvarnabhumi Airport', 'lat': 13.6900, 'lon': 100.7501, 'icon': 'fa-plane'},
    {'name': 'Don Mueang Airport', 'lat': 13.9138, 'lon': 100.6015, 'icon': 'fa-plane'}
]

# Add POIs to the map
for poi in pois:
    folium.Marker(
        location=[poi['lat'], poi['lon']],
        tooltip=poi['name'],
        icon=folium.Icon(color='blue', icon=poi['icon'], prefix='fa')
    ).add_to(poi_layer)


# --- 5. Add Flows and Markers to the Map ---

for index, row in top_od_pairs.iterrows():
    O_coord = (row['O_lat'], row['O_lon'])
    D_coord = (row['D_lat'], row['D_lon'])
    trip_count = row['Trip_Count']
    route_color = row['Color']
    route_id = row['Route_ID']

    # Scale line weight by flow intensity
    flow_intensity = trip_count / max_trip_count
    line_weight = max(MIN_LINE_WEIGHT, flow_intensity * MAX_LINE_WEIGHT)

    popup_text = f"""
    <b>{route_id}</b><br>
    <b>Trip Count: {trip_count:,}</b><br>
    Origin: {row['O_grid_cell']}<br>
    Destination: {row['D_grid_cell']}
    """
    
    # 1. Draw the OD Flow Line
    folium.PolyLine(
        locations=[O_coord, D_coord],
        weight=line_weight,
        color=route_color, 
        opacity=0.8,
        tooltip=f"{route_id} ({trip_count} Trips)"
    ).add_to(flow_layer)

    # 2. Add Origin Marker (O) 
    folium.CircleMarker(
        location=O_coord,
        radius=5,
        color=route_color,
        fill=True,
        fill_color=route_color,
        fill_opacity=1,
        tooltip=f"{route_id} Origin: {row['O_grid_cell']}"
    ).add_to(flow_layer)

    # 3. Add Destination Marker (D) to the MarkerCluster 
    folium.Marker(
        location=D_coord,
        icon=folium.Icon(color='red', icon='fa-bullseye', prefix='fa'), 
        popup=popup_text
    ).add_to(destination_clusters)


# --- 5.5 Add Custom HTML Legend ---

# The legend will use the route IDs and colors, ordered by most frequent route
legend_html = """
     <div style=\"position: fixed; 
                 bottom: 50px; left: 50px; width: 180px; height: auto; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white; opacity:0.9;\">
       &nbsp; <b>Top Routes (Most to Least)</b> <br>
       """
# Add a line for each unique route color and ID, ordered by Trip_Count
for i, row in top_od_pairs.iterrows():
    legend_html += f"""
       &nbsp; <i style='background:{row['Color']}; border-radius:50%; width:10px; height:10px; display:inline-block;'></i> &nbsp; {row['Route_ID']} ({row['Trip_Count']:,} trips)<br>
       """

# Add a section for Line Weight legend
legend_html += f"""
       <hr style=\"margin:5px 0;\">
       &nbsp; <b>Flow Intensity (Weight)</b> <br>
       &nbsp; <i style=\"background:grey; width:10px; height:1px; display:inline-block;\"></i> &nbsp; Low Trips <br>
       &nbsp; <i style=\"background:grey; width:10px; height:5px; display:inline-block;\"></i> &nbsp; Medium Trips <br>
       &nbsp; <i style=\"background:grey; width:10px; height:{MAX_LINE_WEIGHT}px; display:inline-block;\"></i> &nbsp; High Trips <br>
       </div>
       """

m.get_root().html.add_child(folium.Element(legend_html))


# Add a custom HTML title to the map
title_html = f'''
             <h3 align=\"center\" style=\"font-size:18px; font-weight:bold; margin-top:5px; margin-bottom:5px;\">
                Most Frequent BMR <-> BMR Grid-to-Grid Taxi Routes
             </h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# Add a LayerControl to toggle flow, cluster, and POI visibility
folium.LayerControl().add_to(m)

# --- 6. Save to HTML ---
output_filename = "OD_Most_20_BMR_BMR.html"
m.save(output_filename)
print(f"Interactive Folium map saved as {output_filename}")

Interactive Folium map saved as OD_Most_20_BMR_BMR.html
